# One minute files feature extraction

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import csv
import os
import numpy as np
import collections

import scipy.io.wavfile
from scipy.io import loadmat
import collections

import sys, os
sys.path.append(os.path.expanduser('~/projects/engaged_hackathon/'))
from engaged.features import features as engaged_features
from engaged.features import frequency


In [5]:
# getting a list of all the files
base_path = '/home/michael/projects/engaged_hackathon_data/raw_data/one_minute_files'
files = os.listdir(base_path + '/detection_challenge')
files = [xx.split('.')[0] for xx in files]

spectrogram_parameters = {
    'nfft': 1024,
    'window_width': 0.03,
    'overlap': 0.01,
    }

print len(files)

1086


## Creating and saving spectrograms

For each input file, compute and save a spectrogram

In [7]:
spec_path = '/home/michael/projects/engaged_hackathon_data/detection/spectrograms/'
import skimage 


for fname in files:
    print fname    
    
    # load in wav and convert to spectrogram
    sr, wav = scipy.io.wavfile.read(base_path + '/25_Random/' + fname + '.wav')  
    spec, spec_sampl_rate = frequency.spectrogram(wav, sr, **spectrogram_parameters)
    
    # do some smoorthing and denoising  
    spec = skimage.filters.gaussian_filter(spec, 1.0)
    spec -= np.median(spec, axis=1)[:, None]
    spec[spec<0] = 0
    
    # save to disk
    savepath = spec_path + fname + '_spec.mat'
    D = dict(spectrogram=spec, sample_rate=spec_sampl_rate)
    scipy.io.savemat(savepath, D)
    
    # do a reduced size spectrogram
    factor = (75.0/float(spec.shape[0]))
    new_width = int(factor * float(spec.shape[1]))
    output_shape = (75, new_width)
    small_spec = skimage.transform.resize(spec, output_shape)
    
    # save to disk
    savepath = spec_path + fname + '_smallspec.mat'
    D = dict(spectrogram=small_spec, sample_rate=spec_sampl_rate * factor)
    scipy.io.savemat(savepath, D)
    

WC2H8LG-3527_20130709_0758
CR05EF-13527_20130921_0931
SW154LA-3527_20130705_0811
CR05EF-13527_20130917_0331
W84LA-013548_20130622_0300
WC2H8LG-3527_20130710_1013
HA53AA-13548_20130727_0216
E47EN-013527_20131013_1434
W112NN-13548_20130710_1911
SW154LA-3527_20130705_0934
SW112PN-3527_20130823_0634
SW112PN-3527_20130820_1350
SW112PN-3527_20130821_1144
HA86RB-13527_20130724_1723
HA53AA-13548_20130729_1040
E105JP-13548_20131012_2334
WC2H8LG-3527_20130712_0902
CR05EF-13527_20130919_1733
NW1-013527_20130630_1116
SE3-13548_20130907_0511
SW154LA-3527_20130704_2356
W84LA-013548_20130625_1946
CR05EF-13527_20130921_0115
SE23-13527_20130907_0620
HA86RB-13527_20130725_0338
RM14-3YB-944_1_20130616_160300_000
SE3-13548_20130906_1648
SW154LA-3527_20130707_1944
CR8-13548_20130919_0742
W84LA-013548_20130622_0433
NW1-013527_20130624_1917
SW112PN-3527_20130818_1438
W84LA-013548_20130621_1837
CR05EF-13527_20130920_2205
NW1-013527_20130626_1513
W84LA-013548_20130623_1144
CR05EF-13527_20130917_1415
SE3-13548_

## Compare loading times vs generation times

In [12]:
from time import time

number = 10

tic = time()
for fname in files[:number]:
    sr, wav = scipy.io.wavfile.read(base_path + '/25_Random/' + fname + '.wav')  
    spec, spec_sampl_rate = frequency.spectrogram(wav, sr, **spectrogram_parameters)

print "Generation takes %fs" % ((time() - tic) / number)

tic = time()
for fname in files[:number]:
    spec = scipy.io.loadmat(spec_path + fname + '_spec.mat')

print "Loading big takes %fs" %  ((time() - tic) / number)

tic = time()
for fname in files[:number]:
    spec = scipy.io.loadmat(spec_path + fname + '_smallspec.mat')
    
print "Loading small takes %fs" %  ((time() - tic) / number)

print wav.shape

Generation takes 0.316938s
Loading big takes 0.005093s
Loading small takes 0.000522s
(1440008,)


## Choosing locations and saving to disk

For each file, choose a list of where to extract features from

In [16]:
# for each file, compute features for each second and assign to biotic/not_biotic lists
# we will worry about anthropogenic later
# max_files = 50

# maximum num slices of each class to take from each file
max_from_each_file = 500
# files_to_use = files[:max_files]

features_savepath = '/home/michael/projects/engaged_hackathon_data/detection/biotic_anthrop/'


def choose_locations(idxs, maximum, balance=False):
    """
    Given a binary array, the function returns a list of positive and negative locations sampled
    at random from the list.
    Returns 'maximum' locations unless there there are fewer than maximum locations in
    idxs, in which case all are returned
    If balance is true then the classes are balanced to the smaller class size to
    ensure an equal number of each
    """
    false_idxs = np.where(idxs==0)[0]
    true_idxs = np.where(idxs==1)[0]
    
    if false_idxs.shape[0] > maximum and maximum is not None:
        false_idxs = np.random.choice(false_idxs, maximum, replace=False)
        
    if true_idxs.shape[0] > maximum and maximum is not None:
        true_idxs = np.random.choice(true_idxs, maximum, replace=False)
        
    if false_idxs.shape[0] != true_idxs.shape[0] and balance:
        raise Exception("Not implemented!")
        
    return true_idxs, false_idxs
    

for count, fname in enumerate(files):
        
    # load in ground truth - these give positions in terms of the wav file...
    gt = loadmat(base_path + '/detection_challenge/' + fname + '.mat')
      
    # sample positive and negative locations in the file for each sound type
    for soundtype in ['biotic', 'anthropogenic']:
        
        true_idxs, false_idxs = choose_locations(gt[soundtype][0], max_from_each_file)
        idxs = np.hstack([true_idxs, false_idxs])
        labels = np.hstack([np.ones(true_idxs.shape), np.zeros(false_idxs.shape)])
        
        D = dict(true_idxs=true_idxs, false_idxs=false_idxs, 
                 idxs=idxs, labels=labels, sample_rate=gt['sample_rate'])
        savepath = base_path + '/detection_challenge/' + fname + \
            '_sampled_' + soundtype + '.mat'
        scipy.io.savemat(savepath, D)

    if count % 20 == 0:
        print count,

0 20 40 60 80 100 120 140 160 180 200 220 240 260 280 300 320 340 360


In [62]:
def extract_1d_patches(array, locations, hww):
    """
    Extract vertical patches from the array, at the locations given.
    Each slice has a half window width hww
    
    Returns an array of shape:
    (len(locations), array.shape[0], hww*2+1)
    """
    # pad the array to account for overspill
    offset_idxs_np = np.array(locations).ravel() + hww
    extra1 = np.tile(array[:, 0], (hww, 1)).T
    extra2 = np.tile(array[:, -1], (hww, 1)).T
    a_temp = np.hstack((extra1, array, extra2))
    
    # set up the array of index locations to extract from
    idxs = [offset_idxs_np]
    for offset in range(1, hww+1):
        idxs.insert(0, offset_idxs_np-offset)
        idxs.append(offset_idxs_np+offset)
    new_idx = np.vstack(idxs).T.ravel()
    
    # extract the patches and do the appropriate reshapgin
    
    new_shape = (array.shape[0], offset_idxs_np.shape[0], hww*2 + 1)
    to_return = a_temp[:, new_idx].reshape(new_shape).transpose((1, 0, 2))
    return to_return

In [ ]:
# Extracting patches from each spectrogram


In [2]:
# Combining all strips into training and test sets...
# Then each running script can just load in a set of train/test patches

# Do train/test split and combine training data
from sklearn.cross_validation import train_test_split

train_files, test_files = train_test_split(
    range(len(files_to_use)), random_state=0, train_size=0.1, test_size=0.05)

tX, tY = zip(*XY)

data = {}
data['X_train'] = np.vstack([tX[idx] for idx in train_files])
data['y_train'] = np.hstack([tY[idx] for idx in train_files]).astype(np.int32)
data['X_test'] = np.vstack([tX[idx] for idx in test_files])
data['y_test'] = np.hstack([tY[idx] for idx in test_files]).astype(np.int32)

for key in ['X_train', 'X_test']:
    tshape = data[key].shape
    data[key] = data[key].reshape((tshape[0], -1, tshape[1], tshape[2]))
    data[key] = data[key].astype(np.float32)

def balance_classes(X, Y):
    positives = np.where(Y == 0)[0]
    negatives = np.where(Y == 1)[0]
    max_examples = min(len(positives), len(negatives))
    
    if len(positives) > max_examples:
        positives = np.random.choice(positives, max_examples, replace=False)
    if len(negatives) > max_examples:
        negatives = np.random.choice(negatives, max_examples, replace=False)
        
    X = np.vstack((X[negatives, :], X[positives, :]))
    new_Y = np.hstack((Y[negatives], Y[positives])) 
    
    shuffle_idxs = np.random.permutation(X.shape[0])
    
    return X[shuffle_idxs, :], new_Y[shuffle_idxs]
    
# balance the classes...
for key in ['train', 'test']:
    data['X_' + key], data['y_' + key] = balance_classes(data['X_' + key], data['y_' + key])
    
for key, val in data.iteritems():
    print key, val.shape, val.dtype
    print val.min(), val.max(), val.sum()/float(val.shape[0])

for idx in range(10):
    plt.subplot(3, 4, idx+1)
    plt.imshow(data['X_train'][idx, 0, :, :])


NameError: name 'files_to_use' is not defined

## Simple features and substrip pooling features

In [97]:
# set up multithreading - see http://stackoverflow.com/a/16071616/279858

import multiprocessing

def fun(f,q_in,q_out):
    while True:
        i,x = q_in.get()
        if i is None:
            break
        q_out.put((i,f(x)))

def parmap(f, X, nprocs = multiprocessing.cpu_count()):
    q_in   = multiprocessing.Queue(1)
    q_out  = multiprocessing.Queue()

    proc = [multiprocessing.Process(target=fun,args=(f,q_in,q_out)) for _ in range(nprocs)]
    for p in proc:
        p.daemon = True
        p.start()

    sent = [q_in.put((i,x)) for i,x in enumerate(X)]
    [q_in.put((None,None)) for _ in range(nprocs)]
    res = [q_out.get() for _ in range(len(sent))]

    [p.join() for p in proc]

    return [x for i,x in sorted(res)]

In [109]:
features_savepath = '/home/michael/projects/engaged_hackathon_data/detection/features/'


def extract_features(spec, idxs):
    """
    returns a set of features from the spectrogram at points marked by idxs.
    If there are more idx locations than maximum, then will subsample idxs.
    """
    tic = time()    
    
    max_pos = np.argmax(spec[:, idxs], axis=0)[:, None]
    max_val = np.max(spec[:, idxs], axis=0)[:, None]

    substrips = extract_1d_patches(spec[:-1], idxs, 3)
    
    # do the pooling
    substrips_reshape = substrips.reshape((substrips.shape[0], -1, substrips.shape[2]*8))
    substrip_maxs = substrips_reshape.max(2).reshape((idxs.shape[0], -1))
    substrip_vars = substrips_reshape.var(2).reshape((idxs.shape[0], -1))
    substrip_means = substrips_reshape.mean(2).reshape((idxs.shape[0], -1))

    # do the sum of abs differences along the horizontal component...
    abs_diffs_sums = np.abs(np.diff(substrips, axis=2)).sum(axis=2)
    reshaped_abs_diffs = abs_diffs_sums.reshape((substrips.shape[0], -1, 8))
    pooled_abs_diffs = reshaped_abs_diffs.max(2)
    abs_diffs_range = reshaped_abs_diffs.max(2) - reshaped_abs_diffs.min(2)
    
    abs_diffs_max = np.abs(np.diff(substrips, axis=2)).max(axis=2)
    abs_diffs_max_pooled = abs_diffs_max.reshape((substrips.shape[0], -1, 8)).max(2)
    
    return {
        'max_val': max_val,
        'max_pos': max_pos,
        'substrip_maxs': substrip_maxs,
        'substrip_vars': substrip_vars,
        'substrip_means': substrip_means,
        'pooled_abs_diffs': pooled_abs_diffs,
        'abs_diffs_range': abs_diffs_range,
        'abs_diffs_max': abs_diffs_max,
        'abs_diffs_max_pooled': abs_diffs_max_pooled,
        'single_strip': spec[:, idxs].T,
    }


# create the folders in which to save the features
for soundtype in ['biotic', 'anthropogenic']:
    if not os.path.exists(features_savepath + soundtype):
        os.makedirs(features_savepath + soundtype)


def process_file(inputs):

    (count, fname), feature_engine = inputs

    # load the spec
    spec_dict = scipy.io.loadmat(spec_path + fname + '_spec.mat')

    for soundtype in ['biotic', 'anthropogenic']:

        # load the locations
    
    locations = scipy.io.loadmat(
            base_path + '/detection_challenge/' + fname + '_sampled_' + soundtype + '.mat')
        idxs = locations['idxs']

        # convert the locations to the spectrogram sampling rate
        total_wav_samples = (float(locations['sample_rate']) * 60.02)
        total_spec_samples = float(spec_dict['spectrogram'].shape[1])
        rescale_factor = total_spec_samples / total_wav_samples 
        idxs *= rescale_factor

        # create the feature vector
        temp_X = feature_engine(spec_dict['spectrogram'], idxs)

        # save to disk all the features from this one minute file
        for feature_name, feature in temp_X.iteritems():
            savepath = features_savepath + soundtype + '/' + fname + '_' + feature_name + '.mat'
            D = dict(X=feature, Y=locations['labels'], idxs=idxs)
            scipy.io.savemat(savepath, D)

    if count % 5 == 0:
        print str(count) + " ",


tic = time()
args = zip(enumerate(files), [extract_features]*len(files))
parmap(process_file, args)
print time() - tic



 196.103338003
 5   0   15   30   25   35   40   130  10  145  170  55  60  45  50  155  20  185  190  80  75  175  70  230  65  215  255  90  105  180  95  260 85  235  285  120  115  200  110  100  290  315 135  165  210  125  195  325  160  140  275  245  220  355 205  150  280  250  225  265  240  330 350  295  270  300  360 305  335  310 320  345 340 

In [ ]:
# Save spectrogram slices

In [119]:
import IPython
IPython.display.Audio(wav, rate=sr)

In [112]:
def extract_features(spec, idxs):
    """
    returns a set of features from the spectrogram at points marked by idxs.
    If there are more idx locations than maximum, then will subsample idxs.
    """
    tic = time()    
    

    substrips = extract_1d_patches(spec[:-1], idxs, 9)
    print substrips.shape
    
    return {
        'substrips': substrips
    }


def process_file(inputs):

    (count, fname), feature_engine = inputs

    # load the spec
    spec_dict = scipy.io.loadmat(spec_path + fname + '_smallspec.mat')

    for soundtype in ['biotic', 'anthropogenic']:

        # load the locations
        locations = scipy.io.loadmat(
            base_path + '/detection_challenge/' + fname + '_sampled_' + soundtype + '.mat')
        idxs = locations['idxs']

        # convert the locations to the spectrogram sampling rate
        total_wav_samples = (float(locations['sample_rate']) * 60.02)
        total_spec_samples = float(spec_dict['spectrogram'].shape[1])
        rescale_factor = total_spec_samples / total_wav_samples 
        idxs *= rescale_factor

        # create the feature vector
        temp_X = feature_engine(spec_dict['spectrogram'], idxs)

        # save to disk all the features from this one minute file
        for feature_name, feature in temp_X.iteritems():
            savepath = features_savepath + soundtype + '/' + fname + '_' + feature_name + '.mat'
            D = dict(X=feature, Y=locations['labels'], idxs=idxs)
            scipy.io.savemat(savepath, D)

    if count % 5 == 0:
        print str(count) + " ",

        
tic = time()
args = zip(enumerate(files), [extract_features]*len(files))
map(process_file, args)
print time() - tic



(1000, 74, 19)
(1000, 74, 19)
0  (1000, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(500, 74, 19)
(1000, 74, 19)
(500, 74, 19)
(1000, 74, 19)
(500, 74, 19)
(1000, 74, 19)
5  (500, 74, 19)
(1000, 74, 19)
(500, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
10  (500, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(500, 74, 19)
(1000, 74, 19)
(500, 74, 19)
(1000, 74, 19)
(500, 74, 19)
(1000, 74, 19)
15  (500, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(500, 74, 19)
(1000, 74, 19)
(500, 74, 19)
(500, 74, 19)
(500, 74, 19)
(1000, 74, 19)
20  (500, 74, 19)
(1000, 74, 19)
(500, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
25  (500, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(500, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
(1000, 74, 19)
30  (500, 74, 19)
(1000, 74, 19)
(500, 74, 19)
(1000, 74, 19)
(500